## Steps to Accomplish ##

- Download pre-pulled files from Udacity *DONE*
- Pull the file image-predictions.tsv from Udacity's server using the Requests library *DONE*
- Pull Twitter's API to supply the retweet count and favorite count (at minimum) and add that to the data
    - Using the tweet IDs in the twitter archive file, query and store each tweet's JSON data using Tweepy *DONE*
    - Store the entire set of JSON data in a file called tweet_json.txt *DONE*
    - Each tweet's data should be written to its own line *DONE*
    - Read this data line by line in a pandas DataFrame with tweet ID, retweet count, and favorite count at minimum *DONE*
    - Merge this information into the main dataframe *DONE*
- After gathering all of the data, assess them visually and programmatically and find 8 quality and 2 tidiness issues
- Make sure they satisfy the Project Motivation to meet specifications
- Clean all 10 of those issues and document here
- Store the clean DataFrame(s) in a CSV file named twitter_archive_master.csv
- Analyze and visualize the data here
    - There must be at least 3 insights and 1 visualization
- Create a 300-600 word written report called 'wrangle_report.pdf' or 'wrangle_report.html'
    - Use this to describe my wrangling efforts
- Create a 250 word minimum written report called 'act_report.pdf' or 'act_report.html'
    - Use this to communicate the insights and display the visualization from the data

For data cleaning purposes, I need to clean:
- at least 8 quality issues
- at least 2 tidiness issues

And finally:
- REMOVE my Twitter API keys and token from the files before posting

### Tweepy Notes ###

Tweet data is stored in JSON format by Twitter. Getting tweet JSON data via tweet ID using Tweepy is described well in this StackOverflow answer. Note that setting the tweet_mode parameter to 'extended' in the get_status call, i.e., api.get_status(tweet_id, tweet_mode='extended'), can be useful.

Also, note that the tweets corresponding to a few tweet IDs in the archive may have been deleted. Try-except blocks may come in handy here.

Twitter's API has a rate limit. Rate limiting is used to control the rate of traffic sent or received by a server. As per Twitter's rate limiting info page:

Rate limits are divided into 15 minute intervals

To query all of the tweet IDs in the WeRateDogs Twitter archive, 20-30 minutes of running time can be expected. Printing out each tweet ID after it was queried and using a code timer were both helpful for sanity reasons. Setting the wait_on_rate_limit and wait_on_rate_limit_notify parameters to True in the tweepy.api class is useful as well.

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import tweepy
import json

image_predictions.tsv URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

In [2]:
# Establishing the first table: archive

archive = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
# Downloading image-predictions.tsv from Udacity's server

folder_name = "C:/Users/chris/OneDrive/Documents/Coding/Udacity_Data_Analyst/Wrangle_Data_Project"
file_name = 'image-predictions.tsv'
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

response = requests.get(url)
with open(os.path.join(folder_name, file_name), mode='wb') as file:
	file.write(response.content)

In [3]:
# Establishing the second table: image_predictions

image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')

In [4]:
# Setting up the Twitter API

consumer_key = 'REMOVED'
consumer_secret = 'REMOVED'
access_token = 'REMOVED'
access_secret = 'REMOVED'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [5]:
archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
412,822975315408461824,NaN,NaN,2017-01-22 01:12:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Albus. He's soaked as h*ck. Seems to h...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/822975315...,12,10,Albus,None,None,None,None
871,761599872357261312,NaN,NaN,2016-08-05 16:28:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Sephie. According to this picture, she...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/761599872...,11,10,Sephie,None,None,None,None
172,858843525470990336,NaN,NaN,2017-05-01 00:40:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I have stumbled puppon a doggo painting party....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/858843525...,13,10,None,doggo,None,None,None
210,852189679701164033,NaN,NaN,2017-04-12 16:00:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sailor. He has collected the best dirt...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/852189679...,12,10,Sailor,None,None,None,None
1513,691096613310316544,NaN,NaN,2016-01-24 03:14:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we see one dog giving a puptalk to anothe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/691096613...,11,10,None,None,None,None,None


In [6]:
number_of_tweets = archive.tweet_id.count()

In [8]:
tweet_json_file = 'tweet_json.txt'
current_record = 0

for tweet_id in archive.tweet_id:
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        # converting the JSON dictionary object to a string
        tweet_data = json.dumps(tweet._json)
        with open(os.path.join(folder_name, tweet_json_file), mode='a') as file:
            file.write(tweet_data)
            file.write('\n')
        print(str(tweet_id) + 'success')
        current_record += 1
        print(str(current_record / number_of_tweets) * 100) + '% complete')
    except:
        print(str(tweet_id) + 'fail')
        current_record += 1
        print(str((current_record / number_of_tweets) * 100) + '% complete')

892420643555336193success
0.00042444821731748726% complete
892177421306343426success
0.0008488964346349745% complete
891815181378084864success
0.0012733446519524619% complete
891689557279858688success
0.001697792869269949% complete
891327558926688256success
0.0021222410865874364% complete
891087950875897856success
0.0025466893039049238% complete
890971913173991426success
0.0029711375212224107% complete
890729181411237888success
0.003395585738539898% complete
890609185150312448success
0.0038200339558573855% complete
890240255349198849success
0.004244482173174873% complete
890006608113172480success
0.00466893039049236% complete
889880896479866881success
0.0050933786078098476% complete
889665388333682689success
0.0055178268251273345% complete
889638837579907072success
0.005942275042444821% complete
889531135344209921success
0.006366723259762309% complete
889278841981685760success
0.006791171477079796% complete
888917238123831296success
0.007215619694397283% complete
888804989199671297succ

863471782782697472success
0.06239388794567063% complete
863432100342583297success
0.06281833616298811% complete
863427515083354112success
0.0632427843803056% complete
863079547188785154success
0.0636672325976231% complete
863062471531167744success
0.06409168081494057% complete
862831371563274240success
0.06451612903225806% complete
862722525377298433success
0.06494057724957555% complete
862457590147678208success
0.06536502546689305% complete
862096992088072192success
0.06578947368421052% complete
861769973181624320fail
0.06621392190152801% complete
861383897657036800success
0.0666383701188455% complete
861288531465048066success
0.06706281833616298% complete
861005113778896900success
0.06748726655348047% complete
860981674716409858success
0.06791171477079797% complete
860924035999428608success
0.06833616298811546% complete
860563773140209665success
0.06876061120543293% complete
860524505164394496success
0.06918505942275043% complete
860276583193509888success
0.06960950764006792% complet

837482249356513284success
0.12521222410865873% complete
837471256429613056success
0.12563667232597622% complete
837366284874571778fail
0.12606112054329371% complete
837110210464448512success
0.1264855687606112% complete
837012587749474308fail
0.1269100169779287% complete
836989968035819520success
0.1273344651952462% complete
836753516572119041success
0.12775891341256368% complete
836677758902222849success
0.12818336162988114% complete
836648853927522308success
0.12860780984719863% complete
836397794269200385success
0.12903225806451613% complete
836380477523124226success
0.12945670628183362% complete
836260088725786625success
0.1298811544991511% complete
836001077879255040success
0.1303056027164686% complete
835685285446955009success
0.1307300509337861% complete
835574547218894849success
0.13115449915110355% complete
835536468978302976success
0.13157894736842105% complete
835309094223372289success
0.13200339558573854% complete
835297930240217089success
0.13242784380305603% complete
8352

819588359383371776success
0.18803056027164686% complete
819347104292290561success
0.18845500848896435% complete
819238181065359361success
0.18887945670628184% complete
819227688460238848success
0.18930390492359933% complete
819015337530290176success
0.1897283531409168% complete
819015331746349057success
0.19015280135823429% complete
819006400881917954success
0.19057724957555178% complete
819004803107983360success
0.19100169779286927% complete
818646164899774465success
0.19142614601018676% complete
818627210458333184success
0.19185059422750425% complete
818614493328580609success
0.19227504244482174% complete
818588835076603904success
0.1926994906621392% complete
818536468981415936success
0.1931239388794567% complete
818307523543449600success
0.1935483870967742% complete
818259473185828864success
0.19397283531409168% complete
818145370475810820success
0.19439728353140917% complete
817908911860748288success
0.19482173174872666% complete
817827839487737858success
0.19524617996604415% compl

799297110730567681success
0.250848896434635% complete
799063482566066176success
0.25127334465195245% complete
798933969379225600success
0.25169779286926997% complete
798925684722855936success
0.25212224108658743% complete
798705661114773508success
0.25254668930390495% complete
798701998996647937success
0.2529711375212224% complete
798697898615730177success
0.2533955857385399% complete
798694562394996736success
0.2538200339558574% complete
798686750113755136success
0.25424448217317486% complete
798682547630837760success
0.2546689303904924% complete
798673117451325440success
0.25509337860780984% complete
798665375516884993success
0.25551782682512736% complete
798644042770751489success
0.2559422750424448% complete
798628517273620480success
0.2563667232597623% complete
798585098161549313success
0.2567911714770798% complete
798576900688019456success
0.25721561969439727% complete
798340744599797760success
0.2576400679117148% complete
798209839306514432success
0.25806451612903225% complete
79

...

682059653698686977fail
0.7122241086587436% complete
682047327939461121fail
0.7126485568760611% complete
682032003584274432fail
0.7130730050933786% complete
682003177596559360fail
0.7134974533106961% complete
681981167097122816fail
0.7139219015280136% complete
681891461017812993fail
0.7143463497453311% complete
681694085539872773fail
0.7147707979626485% complete
681679526984871937fail
0.715195246179966% complete
681654059175129088fail
0.7156196943972836% complete
681610798867845120fail
0.716044142614601% complete
681579835668455424fail
0.7164685908319185% complete
681523177663676416fail
0.716893039049236% complete
681340665377193984fail
0.7173174872665535% complete
681339448655802368fail
0.717741935483871% complete
681320187870711809fail
0.7181663837011885% complete
681302363064414209fail
0.7185908319185059% complete
681297372102656000fail
0.7190152801358234% complete
681281657291280384fail
0.719439728353141% complete
681261549936340994fail
0.7198641765704584% complete
68124241845329920

676098748976615425fail
0.7797113752122241% complete
676089483918516224fail
0.7801358234295416% complete
675898130735476737fail
0.780560271646859% complete
675891555769696257fail
0.7809847198641766% complete
675888385639251968fail
0.7814091680814941% complete
675878199931371520fail
0.7818336162988115% complete
675870721063669760fail
0.782258064516129% complete
675853064436391936fail
0.7826825127334465% complete
675849018447167488fail
0.783106960950764% complete
675845657354215424fail
0.7835314091680815% complete
675822767435051008fail
0.783955857385399% complete
675820929667219457fail
0.7843803056027164% complete
675798442703122432fail
0.784804753820034% complete
675781562965868544fail
0.7852292020373515% complete
675740360753160193fail
0.7856536502546689% complete
675710890956750848fail
0.7860780984719864% complete
675707330206547968fail
0.7865025466893039% complete
675706639471788032fail
0.7869269949066214% complete
675534494439489536fail
0.7873514431239389% complete
67553147594570956

672594978741354496fail
0.8471986417657046% complete
672591762242805761fail
0.847623089983022% complete
672591271085670400fail
0.8480475382003395% complete
672538107540070400fail
0.8484719864176571% complete
672523490734551040fail
0.8488964346349746% complete
672488522314567680fail
0.849320882852292% complete
672482722825261057fail
0.8497453310696095% complete
672481316919734272fail
0.8501697792869269% complete
672475084225949696fail
0.8505942275042445% complete
672466075045466113fail
0.851018675721562% complete
672272411274932228fail
0.8514431239388794% complete
672267570918129665fail
0.8518675721561969% complete
672264251789176834fail
0.8522920203735145% complete
672256522047614977fail
0.8527164685908319% complete
672254177670729728fail
0.8531409168081494% complete
672248013293752320fail
0.8535653650254669% complete
672245253877968896fail
0.8539898132427843% complete
672239279297454080fail
0.8544142614601019% complete
672231046314901505fail
0.8548387096774194% complete
672222792075620

669661792646373376fail
0.9142614601018676% complete
669625907762618368fail
0.9146859083191851% complete
669603084620980224fail
0.9151103565365025% complete
669597912108789760fail
0.91553480475382% complete
669583744538451968fail
0.9159592529711376% complete
669573570759163904fail
0.916383701188455% complete
669571471778410496fail
0.9168081494057725% complete
669567591774625800fail
0.91723259762309% complete
669564461267722241fail
0.9176570458404074% complete
669393256313184256fail
0.918081494057725% complete
669375718304980992fail
0.9185059422750425% complete
669371483794317312fail
0.9189303904923599% complete
669367896104181761fail
0.9193548387096774% complete
669363888236994561fail
0.919779286926995% complete
669359674819481600fail
0.9202037351443124% complete
669354382627049472fail
0.9206281833616299% complete
669353438988365824fail
0.9210526315789473% complete
669351434509529089fail
0.9214770797962648% complete
669328503091937280fail
0.9219015280135824% complete
669327207240699904f

666776908487630848fail
0.9817487266553481% complete
666739327293083650fail
0.9821731748726655% complete
666701168228331520fail
0.982597623089983% complete
666691418707132416fail
0.9830220713073005% complete
666649482315059201fail
0.983446519524618% complete
666644823164719104fail
0.9838709677419355% complete
666454714377183233fail
0.984295415959253% complete
666447344410484738fail
0.9847198641765704% complete
666437273139982337fail
0.985144312393888% complete
666435652385423360fail
0.9855687606112055% complete
666430724426358785fail
0.9859932088285229% complete
666428276349472768fail
0.9864176570458404% complete
666421158376562688fail
0.9868421052631579% complete
666418789513326592fail
0.9872665534804754% complete
666411507551481857fail
0.9876910016977929% complete
666407126856765440fail
0.9881154499151104% complete
666396247373291520fail
0.9885398981324278% complete
666373753744588802fail
0.9889643463497453% complete
666362758909284353fail
0.9893887945670629% complete
6663532884561018

# Current Status #

Cannot fucking figure out how to read the individual json lines from the API results into a dataframe.

In [33]:
tweet_counts = []

with open('tweet_json.txt', 'r') as file:
    for line in file.readlines():
        this_tweet = json.loads(line)
        this_tweet_data = [this_tweet['id'], this_tweet['retweet_count'], this_tweet['favorite_count']]
        tweet_counts.append(this_tweet_data)

In [34]:
tweet_counts_df = pd.DataFrame(tweet_counts, columns=['tweet_id', 'retweet_count', 'favorite_count'])
tweet_counts_df

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7462,35353
1,892177421306343426,5538,30593
2,891815181378084864,3665,23022
3,891689557279858688,7638,38650
4,891327558926688256,8230,36918
...,...,...,...
872,759159934323924993,1129,0
873,759099523532779520,4061,14249
874,759047813560868866,1964,6375
875,758854675097526272,880,3490


In [35]:
tweet_data = archive.merge(tweet_counts_df, how='left', on='tweet_id')
tweet_data

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,retweet_count,favorite_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,7462.0,35353.0
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,5538.0,30593.0
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,3665.0,23022.0
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,7638.0,38650.0
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,8230.0,36918.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None,NaN,NaN
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None,NaN,NaN
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None,NaN,NaN
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None,NaN,NaN


Sites I used:

Converting JSON dictionary object to string: https://www.geeksforgeeks.org/python-convert-dictionary-object-into-string/

Reading text file line by line: https://tutorial.eyehunts.com/python/python-read-file-line-by-line-readlines/

Converting JSON strings into objects: https://www.freecodecamp.org/news/python-read-json-file-how-to-load-json-from-a-file-and-parse-dumps/